**Table of contents**<a id='toc0_'></a>    
1.1. [Load Data](#toc1_1_)    
1.2. [Basic Summary Statistics](#toc1_2_)    
1.3. [Data Visualization](#toc1_3_)    
1.4. [Assumptions Check](#toc1_4_)    
1.4.1. [Normality](#toc1_4_1_)    
1.4.2. [Robustness](#toc1_4_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [164]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as stats
import seaborn as sns

# force reload modules
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.1. <a id='toc1_1_'></a>[Load Data](#toc0_)

In [165]:
df = pd.read_excel("data/data.xlsx")
df.head()

,Date,LC2A,MP2,MP1,MC
0,22/10/2025 18:00,721.0,870.75,845.12,850.2
1,22/10/2025 20:00,650.0,760.20,848.10,889.8
2,22/10/2025 22:30,644.0,638.40,749.00,682.3
3,23/10/2025 01:00,636.0,552.70,741.00,658.2
4,23/10/2025 04:00,649.0,607.42,755.40,670.2


In [166]:
cols = ["Date", "LC2A", "MP2", "MP1", "MC"]

data = (df[cols]
    .melt(id_vars="Date", var_name="group", value_name="value")
    .dropna()
    .sort_values(by="Date")
    .drop(columns=["Date", "group"])
)
data.head(6)

,value
0,721.00
69,850.20
46,845.12
23,870.75
70,889.80
24,760.20


## 1.2. <a id='toc1_2_'></a>[Basic Summary Statistics](#toc0_)

In [167]:
X = data.values.flatten()
n = 1_000
X = np.random.normal(0, 1, n)
X= np.random.lognormal (1, 1, n)
X = X.flatten()

In [168]:
data = pd.Series(X)
stats_summary:pd.DataFrame = data.describe()
# add skewness and kurtisis to stats_summary
stats_summary.loc["skewness"] = data.skew()
stats_summary.loc["kurtosis"] = data.kurtosis()
stats_summary.round(3)

count       1000.000
mean           4.438
std            6.114
min            0.045
25%            1.245
50%            2.629
75%            5.358
max           87.297
skewness       5.331
kurtosis      47.253
dtype: float64

## 1.3. <a id='toc1_3_'></a>[Data Visualization](#toc0_)

In [169]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import medcouple


def adaptive_boxplot(data, adjusted=False, ax=None):
    """
    Creates an Adjusted Boxplot for skewed distributions (Hubert & Vandervieren, 2008).
    """
    import scipy.stats as stats
    if ax is None:
        fig, ax = plt.subplots()

    X = np.array(data)
    X = X[~np.isnan(X)]  # Remove NaNs
    

    import seaborn as sns
    if not adjusted or (float(abs(stats.skew(X))) < 1 and float(abs(stats.kurtosis(X))) < 1):
        mc = 0
    else:
        mc = medcouple(X)
        
    q1, q3 = np.percentile(X, [25, 75])
    iqr = q3 - q1

    # Calculate whiskers based on medcouple sign
    if mc >= 0:
        lower_whisker = q1 - 1.5 * np.exp(-4 * mc) * iqr
        upper_whisker = q3 + 1.5 * np.exp(3.5 * mc) * iqr
    else:
        lower_whisker = q1 - 1.5 * np.exp(-3.5 * mc) * iqr
        upper_whisker = q3 + 1.5 * np.exp(4 * mc) * iqr

    # Identify outliers
    outliers = X[(X < lower_whisker) | (X > upper_whisker)]

    # Plotting
    # We use a standard boxplot but manually override the whiskers/fliers if needed,
    # or simply plot the components manually for clarity.
    # Here is a manual construction to ensure the adjusted whiskers are respected.

    # Filter X within whiskers for the box and whiskers plot
    mask_within = (X >= lower_whisker) & (X <= upper_whisker)
    adjacent_values = X[mask_within]

    if len(adjacent_values) > 0:
        whisker_min = np.min(adjacent_values)
        whisker_max = np.max(adjacent_values)
    else:
        whisker_min = q1
        whisker_max = q3

    stats = [
        {
            "med": np.median(X),
            "mean": np.mean(X),
            "q1": q1,
            "q3": q3,
            "whislo": whisker_min,
            "whishi": whisker_max,
            "fliers": outliers,
            "label": "",
        }
    ]

    ax.bxp(stats, showfliers=True, orientation="horizontal", showmeans=True, widths=0.4, flierprops={"markersize": 4})
    if mc != 0:
        ax.set_xlabel(f"Adjusted Boxplot (medcouple={mc:.3f}, n_outliers={len(outliers)})", fontsize=10)
    else:    
        ax.set_xlabel(f"n_outliers={len(outliers)}", fontsize=10)
    
    return ax


In [ ]:
import ipywidgets as widgets

def plot_univariate_num(X, hist_stat='count', adjusted=False, transform='boxcox'):
	plt.style.use("seaborn-v0_8-dark")
	fig, ax = plt.subplots(2, figsize=(6,4), height_ratios=(0.8,0.2))
	if transform == 'boxcox':
		X = stats.boxcox(X + 1e-6)[0]  # Box-Cox transformation requires positive data
	elif transform == 'yeojohnson':
		X = stats.yeojohnson(X)[0]  # Yeo-Johnson can handle zero and negative values
	elif transform == 'log':
		X = np.log(X + 1e-6)  # Log transformation requires positive data
	else:
		pass  # No transformation
	sns.histplot(X, kde=True, bins='auto', stat=hist_stat, ax=ax[0])
	ax[0].set_title("Distribution of data")
	ax[0].grid(True)
	adaptive_boxplot(X, ax=ax[1], adjusted=adjusted)
	ax[1].set_xticks([])
widgets.interact(
    plot_univariate_num,
	X=widgets.fixed(X),
	hist_stat=widgets.Dropdown(options=['count', 'probability', 'density'], value='count', description='Hist Stat:'),
	adjusted=widgets.Checkbox(value=True, description="Adjusted Boxplot"),
	transform=widgets.Dropdown(options=['none', 'boxcox', 'yeojohnson', 'log'], value='none', description='Transform:')
);

interactive(children=(Dropdown(description='Hist Stat:', options=('count', 'probability', 'density'), value='c…

## 1.4. <a id='toc1_4_'></a>[Assumptions Check](#toc0_)

In [171]:
from dist_test import check_multimodality, check_normality, recommand_normality_test
from robust_test import recommand_outliers_test, outliers_by_method

In [172]:
%autoreload

is_unimodal, k_modes = check_multimodality(X)
is_unimodal, k_modes


======== GAUSSIAN MIXTURE MODEL (BIC COMPARISON) ========
--------------------------------------------------
Components      BIC                    AIC
--------------------------------------------------
1            6471.84                6462.02               
2            5372.16                5347.62               
3            5064.12                5024.86               
4            5012.60                4958.62               
5            4958.98 ← BEST         4890.27 ← BEST        
--------------------------------------------------
✗ STRONG evidence of MULTIMODALITY (5 components)



(False, 5)

### 1.4.1. <a id='toc1_4_1_'></a>[Normality](#toc0_)

In [173]:
print(f"{'Normality':.^100}")
normality_test_result = []
for r in [0, 5, 11]:
    test = recommand_normality_test(n, percent_outlier=r)
    method = test["alias"]
    verdict = check_normality(X, method)
    normality_test_result.append(verdict)
    print(f"{test['alias']:<15}: {verdict}")

.............................................Normality..............................................
shapiro        : False
normaltest     : False
normaltest     : False


### 1.4.2. <a id='toc1_4_2_'></a>[Robustness](#toc0_)

In [174]:
print(f"{'Outliers':.^100}")
is_normal = normality_test_result[0]
if sum(normality_test_result) == 0 or sum(normality_test_result) == 3:
    method_robustness = recommand_outliers_test(
        X, normality=is_normal, unimodal=is_unimodal
    )
    robustness_result = outliers_by_method(X, method_robustness[0])
else:
    for i in range(6):
        method_robustness = recommand_outliers_test(
            X, normality=is_normal, unimodal=is_unimodal
        )
        robustness_result = outliers_by_method(X, method_robustness[0])
        r = 100 * float(robustness_result["ratio_outliers"])
        test = recommand_normality_test(n, percent_outlier=r)["alias"]
        if check_normality(X, method=test) == is_normal:
            break
        else:
            is_normal = not is_normal
print(method_robustness)
print(robustness_result)

..............................................Outliers..............................................
('LOF', 'DBSCAN')
{'n_outliers': 21, 'ratio_outliers': np.float64(0.021)}


In [163]:
from robust_test import univar_outliers_auto_threshold	

robustness_result = univar_outliers_auto_threshold(X, use_mad=True)
r = 100 * float(robustness_result['ratio_outliers'])
test = recommand_normality_test(n, percent_outlier=r)
method = test["alias"]
verdict = check_normality(X, method)
if verdict == is_normal:
    print(f"Normality: {verdict}, ratio_outliers: {r}%")
else:
    print("explore adjusted boxplot")

Normality: False, ratio_outliers: 8.0%
